# Bootstrapping

In [1]:
from scipy import stats
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

## Sampling with vs. without Replacement

Many questions of probability and statistics have to do with sampling from a population. Sometimes it is appropriate to imagine that the initial population is reset after each draw, and sometimes it is not.

Here's a case of the former: I go around interviewing people and I record their birthdays.

Clearly, two people can have the same birthday, and so I need each birthday to be available for every draw (i.e. interview). This is sampling **with replacement**. (One might even imagine that I interview some people more than once (by random chance).) Note in particular that draws in this context are mutually **independent**.

Here's a case of the latter: I am dealt thirteen cards to make a bridge hand. I can't have the same card appear twice in a single hand, so if I am thinking about the statistics of bridge hands, then I'm thinking about sampling **without replacement**.

Clearly this difference has an effect on correct calculation.

Consider these two similar cases:

**Case 1**: We're playing war. Each of us has a deck of cards, and we each turn over one card at a time, the player with the higher card collecting both. A tie in rank triggers "war", where more cards are laid down and then another contest is initiated on top of the now larger "pot" of cards.

Question: What are the chances that you and I turn over cards with the same rank on one round of war?

Answer: This is effectively a problem of cards drawn *with replacement*: I can just model this with two draws from a single deck. And so I calculate as follows:

I can draw any card first, so that's 52/52. The second card must match the first in rank, so that's 4/52. Thus the chances are $\frac{52}{52}\times\frac{4}{52} = \frac{1}{13}$.

**Case 2**: I am dealt a two-card blackjack hand from a single deck. (Good luck finding this game in Las Vegas!)

Question: What are the chances that I am dealt a pair?

Answer: The two events, one for each card being passed my way, are *not independent*. What the second card is likely to be is affected by what the first card is. For example, if the first card dealt me is the ace of spades, then there is now zero chance that the second card dealt me will be the ace of spades. (Whereas, if the first card is something else, then the chance that the second card be the ace of spades is greater than 0.) And so this is a problem of cards drawn *without replacement*.

So I calculate as follows:

I can draw any card first, so that's 52/52. The second card must match the first in rank, so that's 3/51. There are three left of whatever rank matches my first card, and 51 total cards left. Thus the chances are $\frac{52}{52}\times\frac{3}{51} = \frac{1}{17}$.

Bootstrapping is a sort of sampling ***with replacement***.

I can effect both sorts of sampling with the `choice()` function inside NumPy's `random` module:

In [219]:
X = ['red', 'green', 'blue']
np.random.choice(X)

Let's check the defaults of this function!

In [ ]:
np.random.choice()

## Sampling from an Unknown Distribution

Bootstrapping is used when the shape of the population's distribution is unknown. To simulate this situation, let's make several distributions:

In [153]:
norm = stats.norm(loc=10, scale=5)
expon = stats.expon(loc=5, scale=5)
uni = stats.uniform(loc=5, scale=10)

dists = [norm, expon, uni]

Note that these distributions all have the same mean:

In [220]:
norm.mean() == expon.mean() == uni.mean() == 10

Plotting:

In [155]:
def plot_dists(dists, n=100):
    """Plot histograms of the distributions in dists."""
    fig, axs = plt.subplots(1, len(dists), figsize=(5*len(dists), 5))
    for ax, dist in zip(axs, dists):
        ax.hist(dist.rvs(10000))
        ax.set_xlim(0, 30)
    plt.show()

In [221]:
plot_dists(dists)

Choose a distribution at random:

In [176]:
dist = np.random.choice(dists)

Take a million points from it:

In [177]:
pop = dist.rvs(10**6)

Take a sample of 1000 from that million:

In [178]:
sample = np.random.choice(pop, size=1000)

In [222]:
len(sample)

In [223]:
sample.mean()

The sample mean, $\bar{x}$, is *near* the population mean, $\mu$, but there's a certain gap between them.

## The Algorithm in Words

What we want to do now is what we just did, but *many* times. We can record statistics about each sample, and **then** do statistics on those statistics!

The idea is that statistics on this collection of samples, each made **with replacement**, will be a good approximation of the population parameters from which our original sample was drawn. And the more samples we take, the better our approximation should be. In this way we are "pulling ourselves up by our own bootstraps" to make inferences about the population as a whole.

**Note that we are NOT making inferences about the population distribution, but only about some population parameter of interest (like the mean).**

Let's see what happens when we record the mean and the 95th percentile of each sample:

In [181]:
bootstrap_samples = []                                                   # Initialize an empty list
bootstrap_sample_means = np.zeros(1000)                                  # Initialize an array of means
bootstrap_sample_95pcts = np.zeros(1000)                                 # And another of 95th percentiles
for i in range(1000):
    bootstrap_sample = np.random.choice(sample, size=1000)               # Take 1000 points from one of the dists
    bootstrap_samples.append(bootstrap_sample)                           # Add that to the list
    bootstrap_sample_means[i] = bootstrap_sample.mean()                  # Add the mean to the means' array
    bootstrap_sample_95pct = np.percentile(a=bootstrap_sample, q=95)
    bootstrap_sample_95pcts[i] = bootstrap_sample_95pct                  # Add the 95th percentile to the
                                                                         # percentiles'array

In [224]:
bootstrap_sample_means[:10]

In [225]:
bootstrap_sample_95pcts[:10]

In [226]:
plt.hist(bootstrap_sample_means);

In [227]:
plt.hist(bootstrap_sample_95pcts);

In [228]:
sample.mean()

In [229]:
bootstrap_sample_means.mean()

In [230]:
pop.mean()

In [231]:
bootstrap_sample_means.std()

In [232]:
bootstrap_sample_95pcts.mean()

In [233]:
np.percentile(a=pop, q=95)

## Why Bootstrap?

[Wikipedia](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) is helpful on this.

With a bootstrap we are simulating the relationship between population and sample by treating our sample as the population. In that case we can actually measure the error between our estimates (made through resampling) and the true sample statistics.

"Adèr et al. recommend the bootstrap procedure for the following situations (Adèr, H. J., Mellenbergh G. J., & Hand, D. J. (2008). *Advising on research methods: A consultant's companion*. Huizen, The Netherlands: Johannes van Kessel Publishing. ISBN 978-90-79418-01-5.):

- When the theoretical distribution of a statistic of interest is complicated or unknown. Since the bootstrapping procedure is distribution-independent it provides an indirect method to assess the properties of the distribution underlying the sample and the parameters of interest that are derived from this distribution.
- When the sample size is insufficient for straightforward statistical inference. If the underlying distribution is well-known, bootstrapping provides a way to account for the distortions caused by the specific sample that may not be fully representative of the population.
- When power calculations have to be performed, and a small pilot sample is available. Most power and sample size calculations are heavily dependent on the standard deviation of the statistic of interest. If the estimate used is incorrect, the required sample size will also be wrong. One method to get an impression of the variation of the statistic is to use a small pilot sample and perform bootstrapping on it to get impression of the variance."

## Bootstrapping Real Data

Below we read in a dataset containing information about public toilets in Berlin.

In [237]:
berlin = pd.read_excel('20191101_berlinertoiletten-2.xlsx')

In [234]:
berlin.info()

In [235]:
berlin['Price'].mean()

In [236]:
means = []

for _ in range(10000):
    sample = np.random.choice(berlin['Price'].values, size=len(berlin['Price']))
    means.append(np.mean(sample))

plt.hist(means);

To what extent could we use these results to draw inferences about public toilet prices in all of Germany? Or all of Europe? Or about past or future toilet prices?

## Bootstrapping Challenge

We could use a bootstrap to generate a confidence interval for a hypothesis test.

Suppose we had the following two samples of automobile MPG ratings. The question is whether Group 2 (the experimental group) has a significantly higher MPG rating than Group 1 (the control group). Cars in Group 1 have had an average MPG rating around 24.0; we'll count Group 2's average as significantly higher if it's at least 27.5 MPG.

We don't have very large samples, but we can use a bootstrap to help!

Exercise: Construct 10000 bootstrap samples of the *difference in means* between the two groups. Then order the differences and take the 250th and 9750th values to construct a 95%-confidence interval around our estimate of the difference.

In [22]:
# Read in the files:

grp_1 = pd.read_csv('group1.csv')
grp_2 = pd.read_csv('group2.csv')

In [27]:
# Now bootstrap!

